# Fit a basic model and export to mlflow

In [1]:
import pandas as pd
df = pd.read_csv('../data/BTCUSD_5.csv', header = None, names = ["unix", "open", "high", "low", "close", "volume", "trades"])
df['unix'] = pd.to_datetime(df['unix'], unit='s')
df = df.set_index(pd.DatetimeIndex(df['unix']))
df = df.drop(df[['volume', 'trades', 'unix']], axis=1)
df.sort_values(by='unix', inplace=True)

In [2]:
from ta.momentum import RSIIndicator,StochasticOscillator
df['ema_9'] = df['close'].ewm(9).mean() # exponential moving average of window 9
df['sma_5'] = df['close'].rolling(5).mean() # moving average of window 5
df['sma_20'] = df['close'].rolling(20).mean() # moving average of window 20
EMA_12 = pd.Series(df['close'].ewm(span=12, min_periods=12).mean())
EMA_26 = pd.Series(df['close'].ewm(span=26, min_periods=26).mean())
df['macd'] = pd.Series(EMA_12 - EMA_26)    # calculates Moving Average Convergence Divergence
df['rsi'] = RSIIndicator(df['close']).rsi() # calculates Relative Strength Index 
df['stochastic']=StochasticOscillator(df['high'],df['low'],df['close']).stoch()

In [3]:
# Dropping rows with Na values
df = df.dropna()

## get Y

In [4]:
df['y'] = df['close'].shift(-1)
df = df[:-1]
first = df.index[0].timestamp()
last = df.index[-1].timestamp()

In [5]:
df

,open,high,low,close,ema_9,sma_5,sma_20,macd,rsi,stochastic,y
unix,,,,,,,,,,,
2021-08-07 08:30:00,43902.1,43902.1,43902.1,43902.1,41680.635908,43395.86,40844.785,951.717121,71.691991,100.000000,44158.8
2021-08-07 09:30:00,44158.7,44158.8,44158.7,44158.8,41943.752477,43627.46,41086.915,1003.471642,72.708835,100.000000,43216.9
2021-08-07 10:25:00,43216.9,43216.9,43216.9,43216.9,42078.098180,43562.24,41258.505,964.780071,63.671258,78.894416,44469.4
2021-08-07 13:15:00,43860.9,44469.4,43860.9,44469.4,42329.048442,43899.28,41472.580,1013.660314,69.160687,100.000000,42119.7
2021-08-07 13:20:00,43860.9,43861.0,42119.7,42119.7,42307.186860,43573.38,41583.540,869.395577,52.985389,32.763900,43347.7
...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30 20:45:00,18750.0,18750.0,18750.0,18750.0,19208.110808,18884.24,19292.725,-292.440944,31.942950,0.000000,18676.7
2022-06-30 20:50:00,18676.7,18676.7,18676.7,18676.7,19154.969727,18859.28,19223.150,-300.545148,30.613437,0.000000,18814.6
2022-06-30 21:10:00,18814.6,18814.6,18814.6,18814.6,19120.932755,18855.58,19164.100,-292.469009,36.009512,29.440649,18880.2


# Generate Train Test split

In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
random_state = 42
train_size = 0.8
X = df[['close', 'ema_9', 'sma_5', 'sma_20', 'macd', 'rsi', 'stochastic']]
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size, random_state=random_state)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=train_size, random_state=random_state) 

# Configure MLFlow endpoint

In [7]:
import os
import mlflow

os.environ["AWS_ACCESS_KEY_ID"] = "mlflow"
os.environ["AWS_SECRET_ACCESS_KEY"] = "mlflow123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://mlflow-minio.mlflow.svc.cluster.local:9000/"
model_artifact_name = "model"

mlflow.set_tracking_uri("http://mlflow.mlflow.svc.cluster.local")
mlflow.set_experiment("bitcoin")

<Experiment: artifact_location='s3://mlflow/1', creation_time=None, experiment_id='1', last_update_time=None, lifecycle_stage='active', name='bitcoin', tags={}>

# Fit and export the model

In [13]:
y_hat

array([45066.207, 45068.5  , 27388.447, ..., 19254.4  , 26270.879,
       61485.477], dtype=float32)

# Update the inferenseservice yaml

In [ ]:
#https://github.com/ipython/ipython/issues/6701
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate deployment/isvc.yaml

apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "bitcoin-forecast"
  namespace: default
  labels:
    networking.knative.dev/visibility: cluster-local
spec:
  predictor:
    serviceAccountName: sa-s3
    model:
      modelFormat:
        name: mlflow
      protocolVersion: v2
      storageUri: {storage_uri}

In [ ]:
%%writetemplate ../feast-kserve-transform/deployment/isvc.yaml

apiVersion: "serving.kserve.io/v1beta1"
kind: "InferenceService"
metadata:
  name: "bitcoin-forecast"
  namespace: default
  labels:
    networking.knative.dev/visibility: cluster-local
spec:
  predictor:
    serviceAccountName: sa-s3
    model:
      modelFormat:
        name: mlflow
      protocolVersion: v2
      storageUri: {storage_uri}
  transformer:
    containers:
    - image: piepra/feast-transformer:1.2
      name: btc-transfomer
      command:
      - "python"
      - "app/app.py"
      args:
      - --feast_serving_url
      - feast-feature-server.feast.svc.cluster.local:80
      - --entity_ids
      - "BTC/USD"
      - --feature_refs
      - "crypto_stats:open"
      - "crypto_stats:high"
      - "crypto_stats:low"
      - "crypto_stats:close"
      - --protocol
      - v2

In [ ]:
!curl -v -H "Content-Type: application/json" -d @./payload.json \
 http://bitcoin-forecast.default.svc.cluster.local/v2/models/bitcoin-forecast-basic/infer